# Importando o Dataset

In [1]:
import json

In [2]:
teste = open('Dataset.txt', 'r')
dataset = json.load(teste)

# Chromadb - Armazenando o dataset

In [3]:
import sys
print(sys.executable)

C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [4]:

for i in dataset:
    a = f"""{{
      "date":"{i['date']}",
      "time":"{i['time']}",
      "period":"{i['period']}",
      "day_of_week":"{i['day_of_week']}",
      "season":"{i['season']}",
      "session":"{i['session']}"
      }}
      
      {i["description"]}
      """

    if (i['date'] == '2025-02-12'):
        print(a)

{
      "date":"2025-02-12",
      "time":"07:20:00",
      "period":"manhÃ£",
      "day_of_week":"quarta-feira",
      "season":"verÃ£o",
      "session":"cafÃ©-da-manhÃ£"
      }

      Preparou cafÃ© coado em filtro de papel, observando a primeira luz do dia refletir no lÃ­quido Ã¢mbar enquanto a chaleira assobiava. Um pÃ£o francÃªs amanhecido foi transformado em torradas douradas, untadas com manteiga que derretia em veios translÃºcidos. Do quintal, vinha o barulho de pÃ¡ssaros disputando migalhas caÃ­das da mesa externa, enquanto o cheiro de terra molhada da chuva noturna ainda pairava no ar.
      
{
      "date":"2025-02-12",
      "time":"12:50:00",
      "period":"meio-dia",
      "day_of_week":"quarta-feira",
      "season":"verÃ£o",
      "session":"almoÃ§o"
      }

      Reaqueceu arroz do dia anterior, adicionando ervilhas congeladas que estalavam na panela. Cortou tomates maduros em fatias desiguais, cujo suco se misturou ao azeite no prato formando poÃ§as rosadas. Dura

In [5]:
import ollama
import chromadb
import numpy as np

In [ ]:
client = chromadb.PersistentClient(path="chroma/")
collection = client.get_or_create_collection(name="temporal")

ollama.pull("granite-embedding")

for d, i in enumerate(dataset):
    a = f"""{{
      "date":"{i['date']}",
      "time":"{i['time']}",
      "period":"{i['period']}",
      "day_of_week":"{i['day_of_week']}",
      "season":"{i['season']}",
      "session":"{i['session']}"
      }}
      """
    
    description = i["description"]

    embedding = np.array(ollama.embed(model="granite-embedding:278m", input=a)["embeddings"])
    description_embedding = np.array(ollama.embed(model="granite-embedding:278m", input=description)["embeddings"])

    # Reduzir peso da description em: reduzir 80%
    description_embedding *= 0.2

    # Combinar embeddings
    final_embedding = embedding + description_embedding

    collection.add(
        ids=[str(d)],
        embeddings=final_embedding.tolist(),
        documents=[f"{a}\n\n {description}"]
    )

In [7]:
collection.count()

44

# LLM - Requisição Inicial

In [23]:
pergunta = "O que almocei no dia 2025-02-26?"

In [24]:
embeddingPergunta = ollama.embed(model="granite-embedding:278m", input=pergunta)["embeddings"]
memorias = collection.query(query_embeddings=embeddingPergunta, n_results=6)["documents"]

In [25]:
memorias

[['{\n      "date":"2025-02-26",\n      "time":"20:35:00",\n      "period":"noite",\n      "day_of_week":"terÃ§a",\n      "season":"verÃ£o",\n      "session":"jantar"\n      }\n      \n\n O jantar em casa foi um momento de tranquilidade. A luz suave das velas criava um ambiente Ã\xadntimo, enquanto a mÃºsica clÃ¡ssica preenchia o ar. O sabor do frango assado com ervas pairava no ar, acompanhado por legumes frescos e crocantes. A conversa com a famÃ\xadlia era leve e descontraÃ\xadda, histÃ³rias do dia sendo compartilhadas, risadas e carinho. \n Antes de dormir, um livro nas mÃ£os, a luz suave do abajur iluminando as pÃ¡ginas. A histÃ³ria envolvente me transportava para outro mundo, um refÃºgio da realidade. A sensaÃ§Ã£o de paz e relaxamento me invadia, o corpo cansado, mas a mente tranquila. O sono chegou suavemente, um descanso merecido apÃ³s um dia cheio de experiÃªncias.',
  '{\n      "date":"2025-02-26",\n      "time":"12:20:00",\n      "period":"dia",\n      "day_of_week":"terÃ§a"